<a href="https://colab.research.google.com/github/wajhee-ullah/Image-Classification/blob/main/Image_Classifier_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dataset

In [4]:
from keras.datasets import cifar10
import matplotlib.pyplot as plt
 
(train_X,train_Y),(test_X,test_Y)=cifar10.load_data()

170508288/170498071 [==============================] - 11s 0us/step


Show some images from the training Dataset

In [ ]:
n=6
plt.figure(figsize=(20,10))
for i in range(n):
  plt.subplot(330+1+i)
  plt.imshow(train_X[i])
plt.show()


Import layers and modules for model

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

Convert the pixel value to float and normalize

In [7]:
train_X=train_X.astype('float32')
test_X=test_X.astype('float32')

train_X=train_X/255.0
test_X=test_X/255.0

In [8]:
train_Y=np_utils.to_categorical(train_Y)
test_Y=np_utils.to_categorical(test_Y)
 
num_classes=test_Y.shape[1]

Create Sequential Model and Add Layers

In [9]:
model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(32,32,3),
    padding='same',activation='relu',
    kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Configure Optimizer and Compile

In [10]:
sgd=SGD(lr=0.01,momentum=0.9,decay=(0.01/25),nesterov=False)
 
model.compile(loss='categorical_crossentropy',
  optimizer=sgd,
  metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model Summary

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               4194816   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0

Train Model and test Accuracy

In [12]:
model.fit(train_X,train_Y,
    validation_data=(test_X,test_Y),
    epochs=25,batch_size=32)

_,acc=model.evaluate(test_X,test_Y)
print(acc*100)

Epoch 1/25
1563/1563 [==============================] - 61s 11ms/step - loss: 1.9003 - accuracy: 0.3020 - val_loss: 1.4655 - val_accuracy: 0.4717
Epoch 2/25
1563/1563 [==============================] - 17s 11ms/step - loss: 1.3769 - accuracy: 0.4997 - val_loss: 1.2302 - val_accuracy: 0.5555
Epoch 3/25
1563/1563 [==============================] - 17s 11ms/step - loss: 1.1932 - accuracy: 0.5729 - val_loss: 1.1169 - val_accuracy: 0.6018
Epoch 4/25
1563/1563 [==============================] - 17s 11ms/step - loss: 1.0534 - accuracy: 0.6223 - val_loss: 1.0298 - val_accuracy: 0.6407
Epoch 5/25
1563/1563 [==============================] - 17s 11ms/step - loss: 0.9465 - accuracy: 0.6643 - val_loss: 0.9606 - val_accuracy: 0.6619
Epoch 6/25
1563/1563 [==============================] - 17s 11ms/step - loss: 0.8390 - accuracy: 0.7033 - val_loss: 0.9333 - val_accuracy: 0.6693
Epoch 7/25
1563/1563 [==============================] - 17s 11ms/step - loss: 0.7714 - accuracy: 0.7285 - val_loss: 0.9179 -

Saving Model

In [13]:
model.save("model1_cifar_25epoch.h5")

Making a Prediction

In [ ]:
results={
   0:'aeroplane',
   1:'automobile',
   2:'bird',
   3:'cat',
   4:'deer',
   5:'dog',
   6:'frog',
   7:'horse',
   8:'ship',
   9:'truck'
}
from PIL import Image
import numpy as np
im=Image.open("IMAGE_PATH")
 
im=im.resize((32,32))
im=np.expand_dims(im,axis=0)
im=np.array(im)
pred=model.predict_classes([im])[0]
print(pred,results[pred])